# Digital House - Trabajo Práctico Nº1
## Desafío 1. Análisis exploratorio de un dataset de precios de propiedades
Grupo Nª2
Alumnos:
* Alberto Paparelli
* André Song
* Marianela Riz
* Gloria Caravajal
* Franco Emanuel Rodriguez
* Iván Axel Schweizer

Objetivos:
* Efectuar una limpieza del dataset provisto. Particularmente, deberá diseñar estrategias para lidiar con los datos perdidos en ciertas variables.
* Realizar un análisis descriptivo de las principales variables.
* Crear nuevas columnas a partir de las características dadas que puedan tener valor
predictivo.

In [2]:
# Lo primero que hacemos es importar todas las librerias necesarias.
import numpy as np
import pandas as pd
import requests

In [3]:
# Abrimos el dataset y hacemos un chequeo rápido para saber cantidad de registros y miramos la composición del mismo.
filename = "../dataset/properati.csv"
data = pd.read_csv(filename, on_bad_lines='skip')
display(data.sample(n=6)) 
print(f"(Cantidad de Filas, Cantidad de columnas) -> {data.shape}")

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
107570,107570,sell,house,San Carlos de Bariloche,|Argentina|Río Negro|San Carlos de Bariloche|S...,Argentina,Río Negro,3837856.0,NaN,NaN,...,NaN,2343.750000,NaN,NaN,NaN,NaN,http://www.properati.com.ar/1byda_venta_casa_s...,Chacra de 2 hectarea en el Mallin Ahogado. Bos...,Amarra - Mallin Ahogado,NaN
96495,96495,sell,apartment,Villa Crespo,|Argentina|Capital Federal|Villa Crespo|,Argentina,Capital Federal,3427458.0,NaN,NaN,...,36.0,2568.952381,2997.111111,NaN,NaN,1.0,http://www.properati.com.ar/1biqy_venta_depart...,Tipología 1 ambiente.Ambiente. Balcón. Cocina ...,MONOAMBIENTE EN VENTA.CASSA LUIGGI,https://thumbs4.properati.com/4/89BtaqF55MwXul...
68186,68186,sell,store,Florida,|Argentina|Bs.As. G.B.A. Zona Norte|Vicente Ló...,Argentina,Bs.As. G.B.A. Zona Norte,3433908.0,NaN,NaN,...,3097.0,NaN,710.364869,NaN,NaN,NaN,http://www.properati.com.ar/1aggv_venta_local_...,Edificio comercial- Vicente Lopez - Florida -...,Excelente Edificio comercial sobre Av. Bme. M...,https://thumbs4.properati.com/5/oDnVhAV-p_vWOE...
46043,46043,sell,apartment,Mar del Plata,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3430863.0,"-37.980642,-57.5459964",-37.980642,...,85.0,1130.434783,1529.411765,NaN,3.0,NaN,http://www.properati.com.ar/195iu_venta_depart...,"En el Barrio Perla norte a metros de la costa,...","DUPLEX DE 3 AMBIENTES EN PERLA NORTE, A METR...",https://thumbs4.properati.com/9/I2CvTHFPdo6gh4...
13408,13408,sell,apartment,La Angélica,|Argentina|Bs.As. G.B.A. Zona Norte|Pilar|La A...,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.461507,-58.887684",-34.461507,...,136.0,NaN,900.735294,NaN,3.0,NaN,http://www.properati.com.ar/16p2p_venta_depart...,Corredor Responsable: Lucrecia Saulle - CSI 55...,Venta de Bonito Duplex a Estrenar - TERMINADO!...,https://thumbs4.properati.com/3/vMYmwwgzyv1Ocx...
2414,2414,sell,apartment,Palermo,|Argentina|Capital Federal|Palermo|,Argentina,Capital Federal,3430234.0,"-34.57500839,-58.42057037",-34.575008,...,400.0,NaN,NaN,NaN,3.0,NaN,http://www.properati.com.ar/15k4o_venta_depart...,4 Ambientes + BibliotecaDepartamento de 400 m2...,U$D 1.100.000 - Departamento en Venta - Oro ...,https://thumbs4.properati.com/1/pw_K8k-B4SDJ_1...


(Cantidad de Filas, Cantidad de columnas) -> (121220, 26)


# Columnas
1. `operation`                      -> Tipo de operación (Rent y Sell)
2. `property_type`                  -> Tipo de propiedad (PH, apartment, house y store)
3. `place_name`                     -> Nombre del lugar (Ciudad)
4. `place_with_parent_names`        -> Nombre del lugar + nombre de sus ‘padres’
5. 

COMPLETAR!!!!!

A continuación analizamos cada una de las columnas.

## Columna `operations`
Columna operation: revisamos que opciones tiene, o si se repite lo mismo.

In [4]:
# Columna operation: revisamos que opciones tiene, o si se repite lo mismo.
print(f'Cantidad de registros con NaN: {data.operation.isnull().sum()}')
print("Contamos cantidad de registros unicos:")
print(data.groupby(['operation']).size().reset_index(name='count'))


Cantidad de registros con NaN: 0
Contamos cantidad de registros unicos:
  operation   count
0      sell  121220


### Detalle de análisis de columna `operation`:

Vemos que no tiene NaNs y solo tiene la opción `sell`. Por este motivo la vamos a eliminar.

In [5]:
# Borrado de columna operation. Utilizamos inplace=True para que no imprima el resultado.
data.drop(['operation'], axis=1, inplace=True)

## Columna `proporty_type`
Columna operation: revisamos que información tiene, para ver si las categorias tienen datos sustaciales y no se repiten escritos de manera diferente.

In [6]:
print(f'Cantidad de registros con NaN: {data.property_type.isnull().sum()}')
print("Contamos cantidad de registros unicos:")
print(data.groupby(['property_type']).size().reset_index(name='count'))

Cantidad de registros con NaN: 0
Contamos cantidad de registros unicos:
  property_type  count
0            PH   5751
1     apartment  71065
2         house  40268
3         store   4136


### Detalle de análisis de columna `proporty_type`:

Vemos que no tiene NaNs y tiene 4 categorias `PH`, `apartment`, `house` y `store`.

Todas las categorias tienen una cantidad razonable de datos, por lo que vamos a dejarla sin modificar.

## Columna `place_name`
Columna place_name: Revisamos que opciones tiene, cantidad de registros que se repiten, Nulls, etc.

In [7]:
# Revisamos si existen valores nulos.
print(f'Cantidad de registros con NaN: {data.place_name.isnull().sum()}')

Cantidad de registros con NaN: 23


Hay 23 registros Nulos. Al ser pocos podemos imprimiros para ver si se pueden completar de alguna forma.

Para ello, al ser pocos registros, los vamos a imprimir para analizar.

In [8]:
mask = data.place_name.isnull()
display(data[mask])

,Unnamed: 0,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
6489,6489,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4008968545,-58.63809847",-34.400897,-58.638098,...,240.0,2708.333333,2708.333333,NaN,NaN,NaN,http://www.properati.com.ar/15ui7_venta_casa_t...,"Espectacular casa moderna de una planta, con a...",Venta de casa en Santa María de Tigre,https://thumbs4.properati.com/7/vQjgbHSz1fKXxy...
10201,10201,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.400704,-58.638067",-34.400704,-58.638067,...,300.0,NaN,1783.333333,NaN,5.0,NaN,http://www.properati.com.ar/166p5_venta_casa_t...,Corredor Responsable: SABRINA COCCONI - CSI 62...,VENTA CASA SANTA MARÍA DE TIGRE COUNTRY BARRIO...,https://thumbs4.properati.com/1/FABpKIJMaoAO3t...
11451,11451,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4026880384,-58.6262613379",-34.402688,-58.626261,...,270.0,253.456221,2037.037037,NaN,NaN,NaN,http://www.properati.com.ar/16bpd_venta_casa_t...,MUY LINDA CASA EN EXCELENTE UBICACION!! Superf...,SANTA MARIA DE TIGRE IMPECABLE,https://thumbs4.properati.com/5/fwH9L71hW2rW7V...
14839,14839,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4007994779,-58.6381735719",-34.400799,-58.638174,...,200.0,2000.000000,2400.000000,NaN,NaN,NaN,http://www.properati.com.ar/16tal_venta_casa_b...,Muy linda casa de estilo clásico.-Planta baja;...,Venta de casa en Santa María de Tigre,https://thumbs4.properati.com/1/3ZbPSffTY38y7v...
18622,18622,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4032781195,-58.6263503945",-34.403278,-58.626350,...,340.0,3814.705882,3814.705882,NaN,NaN,NaN,http://www.properati.com.ar/177ee_venta_casa_t...,Casa con increible vista al lago con playa de...,Casa con increible vista al lago con playa y m...,https://thumbs4.properati.com/0/52JCH1PKYCOMe6...
21922,21922,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.399704,-58.638405",-34.399704,-58.638405,...,220.0,NaN,2000.000000,NaN,5.0,NaN,http://www.properati.com.ar/17gy5_venta_casa_s...,Corredor Responsable: Matias Castelli - CUCICB...,Venta Casa - Barrio Santa Maria de Tigre,https://thumbs4.properati.com/8/oO7kmFPYNm8DYr...
23664,23664,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4005161998,-58.6417999185",-34.400516,-58.641800,...,294.0,1632.653061,1632.653061,NaN,NaN,NaN,http://www.properati.com.ar/17mb5_venta_casa_t...,EXCELENTE CASA CLASICA con grandes ventanas y ...,CASA EN VENTA EN SANTA MARIA DE TIGRE CON RENTA,https://thumbs4.properati.com/2/LQCwH6txpFqetn...
24722,24722,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4007292,-58.6377783",-34.400729,-58.637778,...,270.0,1677.215190,1962.962963,NaN,NaN,NaN,http://www.properati.com.ar/17oz1_venta_casa_t...,"EN EXCLUSIVIDAD!!!!!!Lindísima casa, muy lumin...",Venta de casa en Santa María de Tigre,https://thumbs4.properati.com/0/Nhx4NejZaQACap...
38856,38856,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.425087,-58.5796585",-34.425087,-58.579659,...,550.0,NaN,2454.545455,NaN,NaN,NaN,http://www.properati.com.ar/18mcb_venta_casa_t...,SANTA MARIA DE TIGRE- AL RIO - ESPECTACULAR CA...,"Casa (barrio cerrado), Venta",https://thumbs4.properati.com/2/Izq8d5o8ti3kJy...
45970,45970,apartment,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.396074,-58.63738",-34.396074,-58.637380,...,27.0,1727.272727,3518.518519,NaN,1.0,NaN,http://www.properati.com.ar/1958f_venta_depart...,"EDIFICI

Notamos que tenemos todas las geo-localizaciones de los datos faltantes, por lo que vamos a utilizar un endpoint de openstreetmap para obtener la ciudad.

Para ello, creamo funciones auxiliares 

In [9]:
# Funciones Auxiliares
def get_city(display_name):
    """ Función para obtener la ciudad en base al string completo de la ubicación.
    Convertimos el string en una lista, separandolo en base a las comas, y si matchea con el Partido de Tigre
    retornamos la posición anterior de la lista.
    """
    city_list = [x.strip() for x in display_name.split(',')]
    new_name = np.nan

    for index, name in enumerate(city_list):
        if name.strip() == "Partido de Tigre":
            new_name = city_list[index-1]
            break

    return new_name

def get_location_from_geo(lat_lon):
    """Funcion que llama al endpoint de openstreetmap para obtener información en base a la latitud y longitud."""
    url = f"https://nominatim.openstreetmap.org/search.php?q={lat_lon}&polygon_geojson=1&format=json"
    response = requests.get(url=url)
    location = response.json()
    return get_city(location[0]["display_name"])

def complete_city_parent_names(place, city):
    """Función para completar place_with_parent_names con la ciudad."""
    return place[:-1] + city  + place[-1]

Ahora que tenemos las funciones para obtener la ciudad, vamos a buscar las mismas para cada registro del dataset en donde el campo no este y guardamos esta nueva información en el dataset.

In [10]:
# Creamos la mascara
mask = data.place_name.isnull()
# Aplicamos la mascara y por cada item llamamos a la función que se encarga de buscar el nombre faltante.
locations = data[mask]["lat-lon"].apply(get_location_from_geo)
# Guardamos estos nombre es el dataset
data.loc[mask, "place_name"] = locations

Nos dimos cuenta que este nombre tambien falta en el campo `place_with_parent_names` y aprovechando que ya tenemos la mascara y la información, procedemos a actualizarlos.

In [11]:
places = data[mask].apply(lambda x: complete_city_parent_names(x['place_with_parent_names'], x['place_name']), axis=1)
data.loc[mask, "place_with_parent_names"] = places

Ahora que no tenemos valores nulos, vamos a crear una nueva columna, donde pondremos el contenido de `place_name` en los casos en que existan más de 10 propiedades en la ciudad.
En el caso de que la ciudad tenga 10 o menos propiedades, las agruparemos con la categoría `Ciudad Chica`.
La nueva columna se va a llamar: `ciudad_chica`

In [12]:
# Filtro de ciudades con mas de 10 propiedades
list_city_gt_10 = data.groupby(['place_name']).size().reset_index(name='count').query('count > 10').place_name.tolist()
# Se crea la mascara
mask_city_gt_10 = data['place_name'].isin(list_city_gt_10)
print(f"Registros de ciudades con más de 10 propiedades: {mask_city_gt_10.value_counts().loc[True]}")
print(f"Registros de ciudades con 10 o menos propiedades: {mask_city_gt_10.value_counts().loc[False]}")

# se crea la columna nueva ciudad_chica
data['ciudad_chica'] = np.where(data['place_name'].isin(list_city_gt_10), data['place_name'], "Ciudad Chica")

# Chequeo
# 3     Abasto                  139
# 4     Abril Club de Campo     19
# 399   Garuhapé                1
# 1059  Álvarez                 1
display(data[data.place_name.eq("Abasto")][['place_name', 'ciudad_chica']].sample(1))
display(data[data.place_name.eq("Abril Club de Campo")][['place_name', 'ciudad_chica']].sample(1))
display(data[data.place_name.eq("Garuhapé")][['place_name', 'ciudad_chica']].sample(1))
display(data[data.place_name.eq("Álvarez")][['place_name', 'ciudad_chica']].sample(1))

Registros de ciudades con más de 10 propiedades: 119364
Registros de ciudades con 10 o menos propiedades: 1856


,place_name,ciudad_chica
27163,Abasto,Abasto


,place_name,ciudad_chica
66478,Abril Club de Campo,Abril Club de Campo


,place_name,ciudad_chica
94884,Garuhapé,Ciudad Chica


,place_name,ciudad_chica
14018,Álvarez,Ciudad Chica


In [31]:
# Eliminamos espacios en blanco al principio y final de la ciudad
data['place_name'] = data['place_name'].str.strip()
data['ciudad_chica'] = data['ciudad_chica'].str.strip()

## Columna `place_with_parent_names`: 

En esta columna vamos revisar que no existan valores nulos, y probar si hay consistencia entre los datos que tiene y las columnas `place_name`, `country_name` y `state_name`

In [13]:
# Revisamos si existen valores nulos.
print(f'Cantidad de registros con NaN: {data.place_with_parent_names.isnull().sum()}')

Cantidad de registros con NaN: 0


In [36]:
# Convertimos los datos de la columna en una lista
#places_parent = data.apply(lambda x: complete_city_parent_names(x['place_with_parent_names'], x['place_name']), axis=1)
def compare_places(place_with_parent_names, name):
    city = [x.strip() for x in place_with_parent_names.split("|")]
    return True if name in city else False

mask_places_name = data.apply(lambda x: compare_places(x['place_with_parent_names'], x['place_name']), axis=1)
print(f"La con place_name tiene False: Respuesta: {False in mask_places_name.values}")

mask_places_name = data.apply(lambda x: compare_places(x['place_with_parent_names'], x['country_name']), axis=1)
print(f"La con country_name tiene False: Respuesta: {False in mask_places_name.values}")

mask_places_name = data.apply(lambda x: compare_places(x['place_with_parent_names'], x['state_name']), axis=1)
print(f"La con state_name tiene False: Respuesta: {False in mask_places_name.values}")

La con place_name tiene False: Respuesta: False
La con country_name tiene False: Respuesta: False
La con state_name tiene False: Respuesta: False


### Detalle de análisis de columna `place_with_parent_names`:

No hay nada que solucionar, ya que todos los datos estan correctos.
En la columna anterior ya se habían completado los datos faltantes.


--------- ### Template acá abajo ### ---------


--------- ### Template acá abajo ### ---------


--------- ### Template acá abajo ### ---------

## Columna `xxxxxx`
Columna xxx: revisamos xxxxxx

In [15]:
# Que hacemos
# code para analizar la columna

### Detalle de análisis de columna `xxxxxx`:

Lo que encontramos

Lo que vamos a hacer para solucionarlo

In [16]:
# Que hacemos
# code para resolver